In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time

from sqlalchemy import create_engine

## Energy potential: Bhumi (read pdf and figure out terms) & Ryan Cheng (Cleaning the data) US_Renewable_Energy_Technical_Potential

## Old  code for reference only

In [ ]:

# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')
fbi_code = pd.read_csv('input_data/FBI_Code.csv', encoding = "ISO-8859-1")
#fbi_code = pd.read_csv('input_data/FBI_Code.csv',  encoding= 'unicode_escape')
iucr_code = pd.read_csv('input_data/IUCR_Codes.csv', encoding = "ISO-8859-1")

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

#Loading the FBI cvs file
df = fbi_code
df.head()

#Loading the IUCS cvs file
df2 = iucr_code
df2.head(10)

#dropping unnecessary columns
df = df.drop(df.columns[[2,3]], axis=1)
df.columns=['code', 'description']
df.head()

#dropping unnecessary columns
df2 = df2.drop(df2.columns[[3,4]], axis=1)
df2.head()

#Checking and Removing duplicates rows
df = df.drop_duplicates()
df.head()

#writting csv file for import into pgadmin data base
#df.to_csv(r'./Output_data/FBI_code.csv', encoding='utf-8', index=False)

#Checking and Removing duplicates rows
df2 = df2.drop_duplicates()
df2.head()

#Rename Columns
df2 = df2.rename(columns=({'PRIMARY DESCRIPTION':'PRIMARY_DESCRIPTION','SECONDARY DESCRIPTION':'SECONDARY_DESCRIPTION'}))
df2.head()

#writting csv file for import into pgadmin data base
#df2.to_csv(r'./Output_data/ICUS_code.csv', encoding='utf-8', index=False)

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

#dropping unnecessary columns
crimes_data = crimes_data.drop(columns=['Latitude','Location','Longitude','X Coordinate','Y Coordinate','Updated On'])
# crimes_data.shape[0]

# removing duplicate values
crimes_data = crimes_data.drop_duplicates()
# crimes_data.shape[0]

# removing missing values
crimes_data = crimes_data.dropna()
# crimes_data.shape[0]

# Creating a function which will remove extra leading and tailing whitespace from the data.
def whitespace_remover(crimes_data):
   
    # iterating over the columns
    for i in crimes_data.columns:
         
        # checking datatype of each columns
        if crimes_data[i].dtype == 'object':
             
            # applying strip function on column
            crimes_data[i] = crimes_data[i].map(str.strip)
        else:
             
            # if condn. is False then it will do nothing.
            pass
        
# applyting whitespace_remover function on dataframe        
whitespace_remover(crimes_data)

#Rename Columns
crimes_data = crimes_data.rename(columns=({'Case Number':'Case_Number',
                                           'Location Description':'Location_Description',
                                           'Community Area':'Community_Area',
                                          'FBI Code':'FBI_Code'}))


#writting csv file for import into pgadmin data base
#crimes_data.to_csv(r'./Output_data/crimes_data.csv', encoding='utf-8', index=False)
crimes_data.head(20)

# create database connection
connection_string = "postgres:postgressql@localhost:5432/new_crime_db"
engine = create_engine(f'postgresql://{connection_string}')
# Confirm tables
engine.table_names()

# rename columns to lower case 
df2.rename(columns={'IUCR':'iucr','PRIMARY_DESCRIPTION':'primary_description','SECONDARY_DESCRIPTION':'secondary_description' }, inplace=True)
crimes_data.columns=['id', 'case_number', 'date', 'block', 'iucr','location_description',
                     'arrest', 'domestic','beat','district', 'district_name', 'district_population',
                     'ward', 'community_area', 'fbi_code'
                    ]

#load dataframe into database for the FBI and IUCR codes

df2.to_sql(name='iucr_codes', con=engine, if_exists='append', index=False)
df.to_sql(name='fbi_description', con=engine, if_exists='append', index=False)

#load dataframe into database for the crimes data
crimes_data.to_sql(name='crime_table', con=engine, if_exists='append', index=False)

## Energy_consumption: Rafael (TBD) & Ryan Callaghan (TBD) US_Total_Energy_Consumption

## Artem - Webscraping & lats/Longs

In [27]:
# proving url address
url  = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population#cite_note-5"

# sleep for 1 second before reading url
time.sleep(1)

# scrape wiki url
wiki_data = pd.read_html(url)
# type(wiki_data)

# select the first table from the list of tables and convert to DataFrame
population_table = pd.DataFrame(wiki_data[0])

# dropping first level of multi index column headers
population_2021 = population_table.droplevel(0, axis=1) 

# selecting state and population as of 2021 columns
population_2021 = population_2021 [['State or territory','July 1, 2021']]

# rename columns 
population_2021 = population_2021.rename(columns=({'State or territory':'State','July 1, 2021':'Population'}))

# removing extra rows
population_2021 = population_2021[0:52]

# remving territories
population_2021 = population_2021.drop(labels=[29,49], axis=0)
population_2021.head()


,State,Population
0,California,39237836
1,Texas,29527941
2,Florida,21781128
3,New York,19835913
4,Pennsylvania,12964056


## TBD (most likely Ryan + someone else) - Postgress -> push to DB